In [1]:
import seaborn as sns
sns.set_theme(style="whitegrid")
import pandas as pd
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_columns", None)
from classes import Paths

<div class="alert alert-info">

Here you are doing EDA and suddenly you noticed that in some time series in certain classes there is a pronounced difference between the values. But what if this pattern in the time series is an `important feature`? You won't know until you extract this pattern and use it as feature in your model.

<img src="../imgs/03.2.09_1.png" width="1200">

<div class="alert alert-warning">

## **Task**:

<div class="alert alert-info">

The dataset is written to the `df` variable. Generate two new features for each car: `gold_feature_1` and `gold_feature_2`; 
- where the first is the `maximum value of the difference between adjacent dates` (dates are repeated in the dataset), 
- and the second is the `position of this difference`.

Since there may be no difference in the rows, `if gold_feature_1 is less than 5, replace it with zero` (gold_feature_2 does not need to be changed in this case). 
Write the result to the `result` variable.

</div>

<div class="alert alert-info">

- **Remark 1:** The drop position is how the day of this machine is counted (starting from zero), in which the drop is maximum compared to the previous day. The differences in the test are only increasing.  
- **Remark 2:** Consider that the difference value for the first date for each machine is -inf.
</div>

<div class="alert alert-info">

Please note that for one car there may be several trips in one day (or there may not be). Therefore, you can first calculate the average for each day and then drop the duplicates.

In [2]:
paths = Paths()
path_car_train = paths.car_train
path_rides_info = paths.rides_info
path_driver_info = paths.driver_info
path_fix_info = paths.fix_info

In [3]:
train = pd.read_csv(path_car_train)
rides_info = pd.read_csv(path_rides_info)
driver_info = pd.read_csv(path_driver_info)
fix_info = pd.read_csv(path_fix_info)

print("train")
display(train.head(10))
print("rides_info")
display(rides_info.head(10))
print("driver_info")
display(driver_info.head(10))
print("fix_info")
display(fix_info.head(10))

train


,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel
5,b12101843B,Skoda Rapid,economy,petrol,2.36,2013,42176,2018,50.93,engine_ignition
6,Q-9368117S,Nissan Qashqai,standart,petrol,5.32,2012,24611,2014,54.79,engine_overheat
7,O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.26,gear_stick
8,h16895544p,Kia Sportage,standart,petrol,3.50,2014,56384,2017,33.24,gear_stick
9,K77009462l,Smart ForFour,economy,petrol,4.56,2013,41309,2018,39.43,gear_stick


rides_info


,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
0,o52317055h,A-1049127W,b1v,2020-01-01,4.95,21,268,36,113.55,0,514.25,0,1.12,2.91
1,H41298704y,A-1049127W,T1U,2020-01-01,6.91,8,59,36,93.00,1,197.52,0,1.65,4.13
2,v88009926E,A-1049127W,g1p,2020-01-02,6.01,20,315,61,81.96,0,"1,276.33",0,2.60,2.46
3,t14229455i,A-1049127W,S1c,2020-01-02,0.26,19,205,32,128.00,0,535.68,0,3.22,0.91
4,W17067612E,A-1049127W,X1b,2020-01-03,1.21,56,554,38,90.00,1,"1,729.14",0,2.72,-1.82
5,I45176130J,A-1049127W,j1v,2020-01-03,7.52,67,1068,28,36.00,2,363.21,0,0.50,-3.44
6,W11562554A,A-1049127W,A1g,2020-01-04,5.78,30,324,48,61.00,0,"1,314.26",0,1.46,-6.00
7,o13713369s,A-1049127W,B1n,2020-01-04,7.35,29,401,57,65.85,0,"1,753.89",0,0.50,-6.47
8,y62286141d,A-1049127W,h1a,2020-01-05,0.12,64,893,38,114.00,1,"2,022.13",0,-0.16,-5.12
9,V28486769l,A-1049127W,p1e,2020-01-05,3.32,43,424,31,51.30,1,"1,334.57",0,-3.76,-2.08


driver_info


,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
0,27,9.00,865,19.00,l17437965W,1,2019-4-2
1,46,7.90,2116,11.00,Z12362316j,0,2021-11-19
2,59,7.80,947,4.00,g11098715c,0,2021-1-15
3,37,7.00,18,4.00,U12618125q,0,2019-11-20
4,39,8.20,428,7.00,A14375829B,0,2019-7-23
5,21,9.90,831,22.00,L95976611S,1,2020-9-18
6,39,6.90,2293,5.00,z74338505G,0,2022-3-30
7,26,7.90,142,5.00,q11106749z,1,2019-12-22
8,18,9.30,425,18.00,r77865210A,1,2020-6-4
9,23,9.20,601,12.00,t10928335r,1,2020-7-18


fix_info


,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
0,P17494612l,RJ,2020-6-20 2:14,reparking,8.00,49
1,N-1530212S,LM,2020-2-9 20:25,repair,10.00,48
2,B-1154399t,ND,2019-8-24 7:1,reparking,1.00,27
3,y13744087j,PG,2019-8-10 9:29,reparking,1.00,28
4,F12725233R,YC,2020-11-12 5:22,refuel_check,8.00,47
5,O41613818T,RW,2019-2-21 13:25,reparking,1.00,32
6,l-1139189J,PO,2020-3-2 19:11,reparking,1.00,28
7,d-2109686j,ML,2018-3-2 5:12,repair,7.40,39
8,u29695600e,QN,2020-2-2 20:10,reparking,10.00,64
9,U75286923j,KC,2019-9-2 6:32,reparking,1.00,24
